In [1]:
!pip install -U nilearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 43.0 MB/s eta 0:00:00
  Attempting uninstall: nibabel
    Found existing installation: nibabel 3.0.2
    Uninstalling nibabel-3.0.2:
      Successfully uninstalled nibabel-3.0.2


In [2]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import numpy as np
import pandas as pd
from skimage.io import imread
from time import time
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import AxesGrid
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split
from sklearn.model_selection import  cross_val_score
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import GridSearchCV, KFold, cross_validate
from sklearn.linear_model import Perceptron
from sklearn.decomposition import PCA
from nilearn import plotting
import nibabel as nib


from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

import tensorflow as tf

# import tensorflow models
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Input, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import optimizers
print(tf.keras.__version__)

2.12.0


In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [4]:
## Read data
Working_directory="/content/drive/MyDrive/ima205-challenge-2023/" 

In [5]:
#On crée un dictionnaire avec toutes nos images train (originales et segmentée)

data_train = {}
for i in range(1,101):
    images = {}
    
    #ED_seg
    img_path = Working_directory + './Train/' + f'{i:03}' +  '/'+ f'{i:03}' +'_ED_seg.nii'
    images["img_ED_seg"] = nib.load(img_path)
    images["data_ED_seg"] = nib.load(img_path).get_fdata()
    
    #ED
    img_path = Working_directory + './Train/' + f'{i:03}' +  '/'+ f'{i:03}' +'_ED.nii'
    images["img_ED"] = nib.load(img_path)
    images["data_ED"] = nib.load(img_path).get_fdata()
    
    #ES_seg
    img_path = Working_directory + './Train/' + f'{i:03}' +  '/'+ f'{i:03}' +'_ES_seg.nii'
    images["img_ES_seg"] = nib.load(img_path)
    images["data_ES_seg"] = nib.load(img_path).get_fdata()
    
    #ES
    img_path = Working_directory +'./Train/' + f'{i:03}' +  '/'+ f'{i:03}' +'_ES.nii'
    images["img_ES"] = nib.load(img_path)
    images["data_ES"] = nib.load(img_path).get_fdata()
    
    
    data_train[f'{i:03}'] = images
    
    

In [6]:
#On crée un dictionnaire avec toutes nos images test (originales et segmentée)

data_test = {}
for i in range(101,151):
    images = {}
    
    #ED_seg
    img_path = Working_directory + './Test/' + f'{i:03}' +  '/'+ f'{i:03}' +'_ED_seg.nii'
    images["img_ED_seg"] = nib.load(img_path)
    images["data_ED_seg"] = nib.load(img_path).get_fdata()
    
    #ED
    img_path = Working_directory + './Test/' + f'{i:03}' +  '/'+ f'{i:03}' +'_ED.nii'
    images["img_ED"] = nib.load(img_path)
    images["data_ED"] = nib.load(img_path).get_fdata()
    
    #ES_seg
    img_path = Working_directory + './Test/' + f'{i:03}' +  '/'+ f'{i:03}' +'_ES_seg.nii'
    images["img_ES_seg"] = nib.load(img_path)
    images["data_ES_seg"] = nib.load(img_path).get_fdata()
    
    #ES
    img_path = Working_directory + './Test/' + f'{i:03}' +  '/'+ f'{i:03}' +'_ES.nii'
    images["img_ES"] = nib.load(img_path)
    images["data_ES"] = nib.load(img_path).get_fdata()
    
    
    data_test[f'{i:03}'] = images

In [7]:
from scipy import ndimage
from skimage import segmentation
import torch
import matplotlib.colors as mcolors
import cv2
from skimage import measure

In [8]:
for images in data_test.values():
  data_ED_seg = images.get("data_ED_seg")
  data_ES_seg = images.get("data_ES_seg")
  for i in range(0,data_ED_seg.shape[2]):
    if 2 in data_ED_seg[:,:,i]:
      center1, center2 = ndimage.measurements.center_of_mass(data_ED_seg[:,:,i]==2)
      mask = data_ED_seg[:,:,i]
      data_ED_seg[:,:,i] = segmentation.flood_fill(mask, (int(center1), int(center2)), 3)
    if 2 in data_ES_seg[:,:,i]:
      center1, center2 = ndimage.measurements.center_of_mass(data_ES_seg[:,:,i]==2)
      mask = data_ES_seg[:,:,i]
      data_ES_seg[:,:,i] = segmentation.flood_fill(mask, (int(center1), int(center2)), 3)
  images["data_ED_seg"] = data_ED_seg
  images["data_ES_seg"] = data_ES_seg

<ipython-input-8-b8231a7c05bf>:6: DeprecationWarning: Please use `center_of_mass` from the `scipy.ndimage` namespace, the `scipy.ndimage.measurements` namespace is deprecated.
  center1, center2 = ndimage.measurements.center_of_mass(data_ED_seg[:,:,i]==2)
<ipython-input-8-b8231a7c05bf>:10: DeprecationWarning: Please use `center_of_mass` from the `scipy.ndimage` namespace, the `scipy.ndimage.measurements` namespace is deprecated.
  center1, center2 = ndimage.measurements.center_of_mass(data_ES_seg[:,:,i]==2)


In [9]:
def find_circonference(data, layer):
  circonference = 0
  for dim in range(data.shape[2]):
    contour = measure.find_contours(data[:,:,dim]==layer, 0.5) 
    circonference += len(contour)
  return circonference

In [10]:
def extract_feature(dictionnaire, dataframe):
  '''
  This method is the main for finding new features in our datas. 
  @dictionnaire is the data_train or data_test created before.
  @dataframe is the df_train or df_test, dataframe in which we want to add features
  '''
  #ED
  grey_mean_RVC_ED = []
  grey_mean_MYO_ED = []
  grey_mean_LVC_ED = []

  grey_std_RVC_ED = []
  grey_std_MYO_ED = []
  grey_std_LVC_ED = []

  BCKG_Volume_ED = []
  RVC_Volume_ED = []
  MYO_Volume_ED = []
  LVC_Volume_ED = []
  MYO_center_of_mass_cx_ED = []
  MYO_center_of_mass_cy_ED = []
  MYO_center_of_mass_cz_ED = []

  contour_ED_RVC = []
  contour_ED_MYO = []
  contour_ED_LVC = []

  #ES
  grey_mean_RVC_ES = []
  grey_mean_MYO_ES = []
  grey_mean_LVC_ES = []

  grey_std_RVC_ES = []
  grey_std_MYO_ES = []
  grey_std_LVC_ES = []

  BCKG_Volume_ES = []
  RVC_Volume_ES = []
  MYO_Volume_ES = []
  LVC_Volume_ES = []
  MYO_center_of_mass_cx_ES = []
  MYO_center_of_mass_cy_ES = []
  MYO_center_of_mass_cz_ES = []

  contour_ES_RVC = []
  contour_ES_MYO = []
  contour_ES_LVC = []

  ratio_BCKG_Volume = []
  ratio_RVC_Volume = []
  ratio_MYO_Volume = []
  ratio_LVC_Volume = []
  for images in dictionnaire.values():
    #Feature extraction for ED
    data_ED_seg = images.get("data_ED_seg")
    data_ED = images.get("data_ED")

    #Grey mean for each type of pixel.

    only_RVC = np.where(data_ED_seg==1, data_ED, 0)
    only_MYO = np.where(data_ED_seg==2, data_ED, 0)
    only_LVC = np.where(data_ED_seg==3, data_ED, 0)

    grey_mean_RVC_ED.append(np.mean(only_RVC))
    grey_mean_MYO_ED.append(np.mean(only_MYO))
    grey_mean_LVC_ED.append(np.mean(only_LVC))

    grey_std_RVC_ED.append(np.std(only_RVC))
    grey_std_MYO_ED.append(np.std(only_MYO))
    grey_std_LVC_ED.append(np.std(only_LVC))
  

    #pixel distribution for ED

    a=np.sum(np.where(data_ED_seg==0, 1, 0))
    b=np.sum(np.where(data_ED_seg==1, 1, 0))
    c=np.sum(np.where(data_ED_seg==2, 1, 0))
    d=np.sum(np.where(data_ED_seg==3, 1, 0))
    BCKG_Volume_ED.append(a)
    RVC_Volume_ED.append(b)
    MYO_Volume_ED.append(c)
    LVC_Volume_ED.append(d)

    #Center of mass for ED, only for Myocardium
    temp = np.where(data_ED_seg==2, 1, 0)
    cx, cy, cz = ndimage.measurements.center_of_mass(temp, labels = temp, index=1)
    MYO_center_of_mass_cx_ED.append(cx)
    MYO_center_of_mass_cy_ED.append(cy)
    MYO_center_of_mass_cz_ED.append(cz)

    #Contours for ED

    contour_ED_RVC.append(find_circonference(data_ED_seg, layer=1))
    contour_ED_MYO.append(find_circonference(data_ED_seg, layer=2))
    contour_ED_LVC.append(find_circonference(data_ED_seg, layer=3))

    #Feature extraction for ES
    data_ES_seg = images.get("data_ES_seg")
    data_ES = images.get("data_ES")

    #Grey mean for each type of pixel.

    only_RVC = np.where(data_ES_seg==1, data_ES, 0)
    only_MYO = np.where(data_ES_seg==2, data_ES, 0)
    only_LVC = np.where(data_ES_seg==3, data_ES, 0)

    grey_mean_RVC_ES.append(np.mean(only_RVC))
    grey_mean_MYO_ES.append(np.mean(only_MYO))
    grey_mean_LVC_ES.append(np.mean(only_LVC))

    grey_std_RVC_ES.append(np.std(only_RVC))
    grey_std_MYO_ES.append(np.std(only_MYO))
    grey_std_LVC_ES.append(np.std(only_LVC))

    #pixel distribution for ES
    
    e=np.sum(np.where(data_ES_seg==0, 1, 0)) 
    f=np.sum(np.where(data_ES_seg==1, 1, 0))
    g=np.sum(np.where(data_ES_seg==2, 1, 0))
    h=np.sum(np.where(data_ES_seg==3, 1, 0))
    BCKG_Volume_ES.append(e)
    RVC_Volume_ES.append(f)
    MYO_Volume_ES.append(g)
    LVC_Volume_ES.append(h)

    #Center of mass for ES, only for Myocardium

    temp = np.where(data_ES_seg==2, 1, 0)
    cx, cy, cz = ndimage.measurements.center_of_mass(temp, labels = temp, index=1)
    MYO_center_of_mass_cx_ES.append(cx)
    MYO_center_of_mass_cy_ES.append(cy)
    MYO_center_of_mass_cz_ES.append(cz)

    #Contours for ED

    contour_ES_RVC.append(find_circonference(data_ES_seg, layer=1))
    contour_ES_MYO.append(find_circonference(data_ES_seg, layer=2))
    contour_ES_LVC.append(find_circonference(data_ES_seg, layer=3))



    
    #Difference between both distribution
    ratio_BCKG_Volume.append(a/e)
    ratio_RVC_Volume.append(b/f)
    ratio_MYO_Volume.append(c/g) 
    ratio_LVC_Volume.append(d/h)   

  #Grey level
  dataframe = dataframe.assign(grey_mean_RVC_ED = grey_mean_RVC_ED, grey_mean_MYO_ED = grey_mean_MYO_ED, grey_mean_LVC_ED = grey_mean_LVC_ED)
  dataframe = dataframe.assign(grey_mean_RVC_ES = grey_mean_RVC_ES, grey_mean_MYO_ES = grey_mean_MYO_ES, grey_mean_LVC_ES = grey_mean_LVC_ES)
  dataframe = dataframe.assign(grey_std_RVC_ED = grey_std_RVC_ED, grey_std_MYO_ED = grey_std_MYO_ED, grey_std_LVC_ED = grey_std_LVC_ED)
  dataframe = dataframe.assign(grey_std_RVC_ES = grey_std_RVC_ES, grey_std_MYO_ES = grey_std_MYO_ES, grey_std_LVC_ES = grey_std_LVC_ES)

  #pixel distribution
  dataframe = dataframe.assign(background_ED = BCKG_Volume_ED, background_ES = BCKG_Volume_ES)
  dataframe = dataframe.assign(Right_ventricle_cavity_ED = RVC_Volume_ED, Right_ventricle_cavity_ES = RVC_Volume_ES)
  dataframe = dataframe.assign(Myocardium_ED = MYO_Volume_ED, Myocardium_ES = MYO_Volume_ES)
  dataframe = dataframe.assign(left_ventricule_ED = LVC_Volume_ED, left_ventricule_ES = LVC_Volume_ES)
  #Center of mass for Myocardium
  dataframe = dataframe.assign(MYO_center_of_mass_cx_ED = MYO_center_of_mass_cx_ED, MYO_center_of_mass_cx_ES = MYO_center_of_mass_cx_ES)
  dataframe = dataframe.assign(MYO_center_of_mass_cy_ED = MYO_center_of_mass_cy_ED, MYO_center_of_mass_cy_ES = MYO_center_of_mass_cy_ES)
  dataframe = dataframe.assign(MYO_center_of_mass_cz_ED = MYO_center_of_mass_cz_ED, MYO_center_of_mass_cz_ES = MYO_center_of_mass_cz_ES)
  #Contours
  dataframe = dataframe.assign(contour_ED_RVC = contour_ED_RVC, contour_ES_RVC = contour_ES_RVC)
  dataframe = dataframe.assign(contour_ED_MYO = contour_ED_MYO, contour_ES_MYO = contour_ES_MYO)
  dataframe = dataframe.assign(contour_ED_LVC = contour_ED_LVC, contour_ES_LVC = contour_ES_LVC)
  #Ratios
  dataframe = dataframe.assign(ratio_BCKG_Volume = ratio_BCKG_Volume, ratio_RVC_Volume = ratio_RVC_Volume, ratio_MYO_Volume = ratio_MYO_Volume, ratio_LVC_Volume=ratio_LVC_Volume)
  return dataframe

In [11]:
df_train = pd.read_csv(Working_directory + 'metaDataTrain.csv') # reading metaDataTrain
df_test = pd.read_csv(Working_directory + 'metaDataTest.csv') # reading metadataTest

#Extraction of features
df_train = extract_feature(data_train, df_train)
df_test = extract_feature(data_test, df_test)

<ipython-input-10-6fc3055ea3fb>:86: DeprecationWarning: Please use `center_of_mass` from the `scipy.ndimage` namespace, the `scipy.ndimage.measurements` namespace is deprecated.
  cx, cy, cz = ndimage.measurements.center_of_mass(temp, labels = temp, index=1)
<ipython-input-10-6fc3055ea3fb>:129: DeprecationWarning: Please use `center_of_mass` from the `scipy.ndimage` namespace, the `scipy.ndimage.measurements` namespace is deprecated.
  cx, cy, cz = ndimage.measurements.center_of_mass(temp, labels = temp, index=1)
<ipython-input-10-6fc3055ea3fb>:86: DeprecationWarning: Please use `center_of_mass` from the `scipy.ndimage` namespace, the `scipy.ndimage.measurements` namespace is deprecated.
  cx, cy, cz = ndimage.measurements.center_of_mass(temp, labels = temp, index=1)
<ipython-input-10-6fc3055ea3fb>:129: DeprecationWarning: Please use `center_of_mass` from the `scipy.ndimage` namespace, the `scipy.ndimage.measurements` namespace is deprecated.
  cx, cy, cz = ndimage.measurements.center_

In [12]:
X = df_train.loc[:,'Height':'ratio_LVC_Volume'].values # Features
X_test = df_test.loc[:,'Height':'ratio_LVC_Volume'].values # Features
y = df_train['Category']

y = tf.one_hot(y, 5)


In [14]:
def MLP_fit(X_train, y_train,learning_rate = 0.0001, n_hidden = 200, n_epochs = 400):
  # Layer sizes
  n_input = X_train.shape[1]  # Number of input features
  #n_hidden = n_hidden  # Number of hidden nodes

  # create the model (using the sequential API)

  model_mlp = Sequential()   # FILL IN STUDENTS
  model_mlp.add(Dense(n_hidden, activation='relu'))
  model_mlp.add(BatchNormalization())
  model_mlp.add(Dense(n_hidden, activation='relu'))
  model_mlp.add(BatchNormalization())
  model_mlp.add(Dense(n_hidden, activation='relu'))
  model_mlp.add(BatchNormalization())
  model_mlp.add(Dense(n_hidden, activation='relu'))
  model_mlp.add(BatchNormalization())
  model_mlp.add(Dense(5, activation='softmax'))

  # create the loss and optimiser
  learning_rate = learning_rate
  model_mlp.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=learning_rate),metrics=["accuracy"]) # TO FILL IN

  # Run optimisation algorithm
  batch_size = 56

  print('Training')
  model_mlp.fit(X_train, y_train, epochs=n_epochs,batch_size=batch_size) # TO FILL IN
  return model_mlp

In [15]:
X = df_train.loc[:,'Height':'ratio_LVC_Volume'].values # Features
X_test = df_test.loc[:,'Height':'ratio_LVC_Volume'].values # Features
y = df_train['Category']

In [16]:
X_train, X_validate, y_train, y_validate = train_test_split(X, y, train_size = 0.7, random_state=42, shuffle = True)

y_train = tf.one_hot(y_train, 5)
y_validate = tf.one_hot(y_validate, 5)

model_mlp = MLP_fit(X_train, y_train)

print("Validating")
model_mlp.evaluate(X_validate,  y_validate,verbose=2)

Training
Epoch 1/400
2/2 [==============================] - 3s 24ms/step - loss: 0.9272 - accuracy: 0.1286
Epoch 2/400
2/2 [==============================] - 0s 12ms/step - loss: 0.8453 - accuracy: 0.1714
Epoch 3/400
2/2 [==============================] - 0s 12ms/step - loss: 0.7876 - accuracy: 0.2714
Epoch 4/400
2/2 [==============================] - 0s 12ms/step - loss: 0.7247 - accuracy: 0.3143
Epoch 5/400
2/2 [==============================] - 0s 12ms/step - loss: 0.6991 - accuracy: 0.3143
Epoch 6/400
2/2 [==============================] - 0s 11ms/step - loss: 0.6635 - accuracy: 0.4857
Epoch 7/400
2/2 [==============================] - 0s 12ms/step - loss: 0.6325 - accuracy: 0.5286
Epoch 8/400
2/2 [==============================] - 0s 11ms/step - loss: 0.6245 - accuracy: 0.5714
Epoch 9/400
2/2 [==============================] - 0s 16ms/step - loss: 0.5864 - accuracy: 0.7000
Epoch 10/400
2/2 [==============================] - 0s 12ms/step - loss: 0.5777 - accuracy: 0.7714
Epoch 11/4

[3.370262622833252, 0.8333333134651184]

In [17]:
'''#On teste l'overfitting sur nos données
accuracy_train = []
accuracy_validate = []
x = np.linspace(0.2, 0.9, 1)
for i in x:
  X_train, X_validate, y_train, y_validate = train_test_split(X, y, train_size = i, random_state=42, shuffle = True)
  
  y_train = tf.one_hot(y_train, 5)
  y_validate = tf.one_hot(y_validate, 5)

  model_mlp = MLP_fit(X_train, y_train)
  accuracy_train.append(model_mlp.evaluate(X_train, y_train, verbose=2)[1])
  accuracy_validate.append(model_mlp.evaluate(X_validate,  y_validate,verbose=2)[1])
plt.plot(x, accuracy_train, color='green')
plt.plot(x, accuracy_validate, color='red')
plt.show()'''


"#On teste l'overfitting sur nos données\naccuracy_train = []\naccuracy_validate = []\nx = np.linspace(0.2, 0.9, 1)\nfor i in x:\n  X_train, X_validate, y_train, y_validate = train_test_split(X, y, train_size = i, random_state=42, shuffle = True)\n  \n  y_train = tf.one_hot(y_train, 5)\n  y_validate = tf.one_hot(y_validate, 5)\n\n  model_mlp = MLP_fit(X_train, y_train)\n  accuracy_train.append(model_mlp.evaluate(X_train, y_train, verbose=2)[1])\n  accuracy_validate.append(model_mlp.evaluate(X_validate,  y_validate,verbose=2)[1])\nplt.plot(x, accuracy_train, color='green')\nplt.plot(x, accuracy_validate, color='red')\nplt.show()"

In [18]:
y_pred_test = model_mlp.predict(X_test)
y_pred_test = tf.argmax(y_pred_test, axis=1)

2/2 [==============================] - 0s 5ms/step


In [19]:
dictionnary = {'Id' : np.arange(101,151,1), 'Category' : y_pred_test}

In [20]:
SampleSubmission = pd.DataFrame(dictionnary)
SampleSubmission.to_csv('SampleSubmission.csv', index=False)